In [5]:
from sklearn import tree
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef, balanced_accuracy_score
import time

In [6]:
# load ROOT
import ROOT

Welcome to JupyROOT 6.30/06


In [7]:
# load TNeutron class
ROOT.gSystem.Load('/home/user/data/ML/lib/v6_30/TNeutron_cc.so')

0

In [8]:
# set the path to our simulation file
#datapath = "/home/user/data/ML/MLP_mult2/"
#datapath = "/home/user/data/ML/e15118/"
datapath = "/home/user/data/ML/stmona/"

In [9]:
# open the ROOT file and load the TTree that it contains
#rootfile = ROOT.TFile(datapath + "delta18_O262plus_2nThermal2.root","READ")
# try this one for training
rootfile = ROOT.TFile(datapath + "tneutron_mult2_26O_24O+2n-uniform.root","READ")
# try this one to test your trained model
rootfile1 = ROOT.TFile(datapath + "tneutron_mult2_26O_24O+2n-delta500keV.root","READ")
tre1= rootfile1.Get("snt")
tre = rootfile.Get("snt")
#tree.Print()

In [10]:
# copy data from the tree
entries = tre.GetEntries();
hit_data = np.zeros((entries, 10));
label = np.empty(entries);
label.fill(9);
for i in range(entries):
  # get the tree index for the next entry on our list
  # and load this entry from the TTree
  tre.GetEntry(i)
  # transfer the data for this entry to our numpy array
  # x0
  hit_data[i][0] = tre.g.x[0]
  # y0
  hit_data[i][1] = tre.g.y[0]
  # z0
  hit_data[i][2] = tre.g.z[0]
  # t0
  hit_data[i][3] = tre.g.t[0]
  # q0
  hit_data[i][4] = tre.g.q[0]
  # x1
  hit_data[i][5] = tre.g.x[1]
  # y1
  hit_data[i][6] = tre.g.y[1]
  # z1
  hit_data[i][7] = tre.g.z[1]
  # t1
  hit_data[i][8] = tre.g.t[1]
  # q1
  hit_data[i][9] = tre.g.q[1]
  # label
  label[i]= tre.signal
  #label[i]= tree.target

In [11]:
# create calculated data matrix
entries = tre.GetEntries();
hit_calc= np.zeros((entries, 7));
for i in range(entries):
  # calculated quantities
  tre.GetEntry(i);
  hit_calc[i][0] = tre.g.GetVelocity(0);
  hit_calc[i][1] = tre.g.GetVelocity(1);
  hit_calc[i][2] = tre.g.HitSeparation(0,1);
  hit_calc[i][3] = tre.g.HitVdiff(0,1);
  hit_calc[i][4] = tre.g.HitOpeningAngle(0,1);
  hit_calc[i][5] = tre.g.HitNSI(0,1);
  hit_calc[i][6] = tre.g.HitScatteringAngle(0,1);

In [12]:
# copy data from the tree
entries1 = tre1.GetEntries();
hit_data1 = np.zeros((entries1, 10));
label1 = np.empty(entries1);
label1.fill(9);
for o in range(entries1):
  # get the tree index for the next entry on our list
  # and load this entry from the TTree
  tre1.GetEntry(o)
  # transfer the data for this entry to our numpy array
  # x0
  hit_data1[o][0] = tre1.g.x[0]
  # y0
  hit_data1[o][1] = tre1.g.y[0]
  # z0
  hit_data1[o][2] = tre1.g.z[0]
  # t0
  hit_data1[o][3] = tre1.g.t[0]
  # q0
  hit_data1[o][4] = tre1.g.q[0]
  # x1
  hit_data1[o][5] = tre1.g.x[1]
  # y1
  hit_data1[o][6] = tre1.g.y[1]
  # z1
  hit_data1[o][7] = tre1.g.z[1]
  # t1
  hit_data1[o][8] = tre1.g.t[1]
  # q1
  hit_data1[o][9] = tre1.g.q[1]
  # label
  label1[o]= tre1.signal

In [13]:
# create calculated data matrix
entries1 = tre1.GetEntries();
hit_calc1= np.zeros((entries1, 7));
for o in range(entries1):
  # calculated quantities
  tre1.GetEntry(o);
  hit_calc1[o][0] = tre1.g.GetVelocity(0);
  hit_calc1[o][1] = tre1.g.GetVelocity(1);
  hit_calc1[o][2] = tre1.g.HitSeparation(0,1);
  hit_calc1[o][3] = tre1.g.HitVdiff(0,1);
  hit_calc1[o][4] = tre1.g.HitOpeningAngle(0,1);
  hit_calc1[o][5] = tre1.g.HitNSI(0,1);
  hit_calc1[o][6] = tre1.g.HitScatteringAngle(0,1);

In [14]:
# Concatenate measured and calculated quantities
input_data_min_max1 = np.concatenate((hit_data1, hit_calc1), axis=1)
X1 = input_data_min_max1
y1 = label1
# Scale the features using Standered
scaler = StandardScaler()
Dataset_featured1 = scaler.fit_transform(X1)

In [15]:
# Concatenate measured and calculated quantities
input_data_min_max = np.concatenate((hit_data, hit_calc), axis=1)
X = input_data_min_max
y = label

# Scale the features using Standered
Dataset_featured = scaler.fit_transform(X)

In [16]:
clf = tree.DecisionTreeClassifier()

In [40]:
number = []
timearray = []
accuracy_list = []
bas_list = []
F1_list = []
Mc_list = []

for i in range(100, 20000, 500):
    start = time.time()
    selection_X1 = Dataset_featured[0:i, :]
    selection_Y1 = y[0:i]

    clf.fit(selection_X1, selection_Y1)
    y_pred = clf.predict(Dataset_featured1)
    end = time.time()
    length = end - start
    
    print(set(y1))
    print(set(y_pred))
    y1_series = pd.Series(y1)
    y_pred_series = pd.Series(y_pred)
    print(y1_series.unique())
    print(y_pred_series.unique())


    accuracy = accuracy_score(y1, y_pred)
    Mc = matthews_corrcoef(y1, y_pred)
    F1 = f1_score(y1, y_pred, average='micro')
    bas = balanced_accuracy_score(y1, y_pred)


    timearray.append(length)
    number.append(i)
    accuracy_list.append(accuracy)
    bas_list.append(bas)
    F1_list.append(F1)
    Mc_list.append(Mc)

    print("Accuracy:", accuracy)
    print("It took", length, "seconds!", i)

print(number)
print(timearray)
print("Accuracy:", accuracy_list)
print("F1 Accuracy:", F1_list)
print("bas Accuracy:", bas_list)
print("Mc Accuracy:", Mc_list)

plt.plot(number, accuracy_list, label="accuracy")
plt.plot(number, F1_list, label="F1")
plt.plot(number, bas_list, label="bas")
plt.plot(number, Mc_list, label="Mc")

plt.xlabel('number of data points')
plt.ylabel('accuracy')
plt.title('Accuracy')
plt.legend()
        

{0.0, 1.0}
{0.0, 1.0}
[1. 0.]
[0. 1.]


Accuracy: 0.663953541752743
It took 0.08322644233703613 seconds! 100
{0.0, 1.0}
{0.0, 1.0}
[1. 0.]
[1. 0.]


Accuracy: 0.7140568333103796
It took 0.025999784469604492 seconds! 600
{0.0, 1.0}
{0.0, 1.0}
[1. 0.]
[1. 0.]


Accuracy: 0.7474039388513979
It took 0.02896904945373535 seconds! 1100
{0.0, 1.0}
{0.0, 1.0}
[1. 0.]
[1. 0.]


Accuracy: 0.5142037368590185
It took 0.03746294975280762 seconds! 1600
{0.0, 1.0}
{0.0, 1.0}
[1. 0.]
[1. 0.]


...

Accuracy: 0.535807739980719
It took 0.5368776321411133 seconds! 18600


{0.0, 1.0}
{0.0, 1.0}
[1. 0.]
[1. 0.]


Accuracy: 0.5256943488041133
It took 0.6889338493347168 seconds! 19100


{0.0, 1.0}
{0.0, 1.0}
[1. 0.]
[1. 0.]


Accuracy: 0.5413808933572052
It took 0.5153753757476807 seconds! 19600
[100, 600, 1100, 1600, 2100, 2600, 3100, 3600, 4100, 4600, 5100, 5600, 6100, 6600, 7100, 7600, 8100, 8600, 9100, 9600, 10100, 10600, 11100, 11600, 12100, 12600, 13100, 13600, 14100, 14600, 15100, 15600, 16100, 16600, 17100, 17600, 18100, 18600, 19100, 19600]
[0.08322644233703613, 0.025999784469604492, 0.02896904945373535, 0.03746294975280762, 0.05063986778259277, 0.055877685546875, 0.07206439971923828, 0.076904296875, 0.08759856224060059, 0.10330796241760254, 0.11113500595092773, 0.12987589836120605, 0.1417090892791748, 0.17165017127990723, 0.1597905158996582, 0.16701054573059082, 0.18401598930358887, 0.18906188011169434, 0.2052135467529297, 0.25151753425598145, 0.22946977615356445, 0.23468899726867676, 0.24975299835205078, 0.26549816131591797, 0.31172823905944824, 0.35024476051330566, 0.3349466323852539, 0.33131980895996094, 0.38683223724365234, 0.37598371505737305, 0.3625926971435547, 0.37890195846557617, 0.399164

In [37]:
feature_names=["x0","y0","z0","t0","q0","x1","y1","z1","t1","q1","Velocity","Velocity1","HitSep","HitVdiff","HOA","HNSI","HSA"]
labels = ["background","signal"]

In [38]:
plt.figure(figsize=(30,10), facecolor ='k')

a = tree.plot_tree(clf,feature_names = feature_names, class_names =labels, rounded = True, filled = True, fontsize=14)

plt.show()


from sklearn.tree import export_text

tree_rules = export_text(clf,feature_names = list(feature_names))

print(tree_rules)

|--- HNSI <= -0.15
|   |--- q1 <= -0.74
|   |   |--- t0 <= -0.60
|   |   |   |--- class: 1.0
|   |   |--- t0 >  -0.60
|   |   |   |--- x1 <= 0.06
|   |   |   |   |--- HNSI <= -0.23
|   |   |   |   |   |--- x0 <= -0.14
|   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |--- x0 >  -0.14
|   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |--- HNSI >  -0.23
|   |   |   |   |   |--- class: 0.0
|   |   |   |--- x1 >  0.06
|   |   |   |   |--- class: 0.0
|   |--- q1 >  -0.74
|   |   |--- q0 <= -0.96
|   |   |   |--- HNSI <= -0.19
|   |   |   |   |--- z1 <= -0.82
|   |   |   |   |   |--- class: 0.0
|   |   |   |   |--- z1 >  -0.82
|   |   |   |   |   |--- class: 1.0
|   |   |   |--- HNSI >  -0.19
|   |   |   |   |--- class: 0.0
|   |   |--- q0 >  -0.96
|   |   |   |--- class: 1.0
|--- HNSI >  -0.15
|   |--- HitSep <= 0.55
|   |   |--- HOA <= -0.26
|   |   |   |--- HNSI <= 0.81
|   |   |   |   |--- HitSep <= 0.12
|   |   |   |   |   |--- HNSI <= -0.10
|   |   |   |   |   |   |--- 

In [39]:
import seaborn as sns
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y1, y_pred)

matrix_df = pd.DataFrame(confusion_matrix)

ax = plt.axes()

sns.set(font_scale=1.3)

plt.figure(figsize=(10,7))

sns.heatmap(matrix_df, annot=True, fmt="g", ax=ax, cmap="magma")

ax.set_title('Confusion Matrix - Decision Tree')

ax.set_xlabel("Predicted label", fontsize =15)

ax.set_xticklabels([]+labels)

ax.set_ylabel("True Label", fontsize=15)

ax.set_yticklabels(list(labels), rotation = 0)

plt.show()

<Figure size 720x504 with 0 Axes>